In [5]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

import matplotlib.pyplot as plt
%matplotlib inline

from InteriorHelmholtzSolver3D import *
from Mesh import *

frequency = 20.0 # frequency [Hz]
k = frequencyToWavenumber(frequency)

(aVertices,_,_) = readVertexArray('SimpleSpeaker')
(aTriangles, aBoundaryMarkers) = readTriangleArray('SimpleSpeaker')
flipOrientation(aTriangles)

interiorPoints = np.array([[0.500, 0.500, 0.500], # in center of speaker interior
                           [0.750, 0.500, 0.500], # upward of center
                           [0.500, 0.750, 0.500], # to the right of center
                           ], dtype=np.float32)


solver = InteriorHelmholtzSolver3D(aVertices, aTriangles)

# Set boundary condition based on boundary marker-values
boundaryCondition = BoundaryCondition(solver.aElement.shape[0])
for i in range(aTriangles.shape[0]):
    if aBoundaryMarkers[i] == 1:
        boundaryCondition.alpha[i] = 0.0
        boundaryCondition.beta[i]  = 1.0
        boundaryCondition.f[i] = 0.0
    elif aBoundaryMarkers[i] == 2:
        boundaryCondition.alpha[i] = 0.0
        boundaryCondition.beta[i]  = 1.0
        boundaryCondition.f[i] = 1.0
    else:
        assert False, "Invalid boundary marker: {}".format(aBoundaryMarkers[i])
        
        
boundaryIncidence = BoundaryIncidence(solver.aElement.shape[0])
boundaryIncidence.phi.fill(0.0)
boundaryIncidence.v.fill(0.0)

interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=np.complex64)

boundarySolution = solver.solveBoundary(k, boundaryCondition, boundaryIncidence)

interiorPhi = solver.solveInterior(boundarySolution, interiorIncidentPhi, interiorPoints)

print "Test Problem 1"
print "==============\n"
print boundarySolution
printInteriorSolution(boundarySolution, interiorPhi)


Test Problem 1

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 0.365301471348 (20.0 Hz)

index          Potential                   Pressure                    Velocity              Intensity
    1  -3.4315e-01+ -1.5539e-01i    2.3530e+01+ -5.1961e+01i    0.0000e+00+  0.0000e+00i     0.0000e+00
    2  -3.2125e-01+ -1.7233e-01i    2.6096e+01+ -4.8646e+01i    0.0000e+00+  0.0000e+00i     0.0000e+00
    3  -1.0468e-01+ -7.2369e-02i    1.0959e+01+ -1.5851e+01i    0.0000e+00+  0.0000e+00i     0.0000e+00
    4  -3.6575e-01+ -1.6394e-01i    2.4825e+01+ -5.5384e+01i    0.0000e+00+  0.0000e+00i     0.0000e+00
    5  -3.5914e-01+ -1.5840e-01i    2.3985e+01+ -5.4383e+01i    0.0000e+00+  0.0000e+00i     0.0000e+00
    6  -2.7617e-01+ -1.1430e-01i    1.7308e+01+ -4.1819e+01i    0.0000e+00+  0.0000e+00i     0.0000e+00
    7  -2.7944e-01+ -1.1169e-01i    1.6913e+01+ -4.2314e+01i    0.0000e+00+  0.0000e+00i     0.0000e+00
    8  -1.1423e-01+ -8.1838e-02i